<h1><center>Import Pre-Trained Word2Vec Model</center></h1>

In [8]:
#pip install gensim
from gensim.models import Word2Vec, KeyedVectors

import pandas as pd
pd.set_option('max_colwidth',99999)
import numpy as np
import re 

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

import gensim
from gensim.models import Word2Vec, word2vec
from gensim.test.utils import common_texts, get_tmpfile

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package stopwords to /Users/wang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/wang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# read in google word2vec pre-trained model
# https://code.google.com/archive/p/word2vec/
model = KeyedVectors.load_word2vec_format('./Data/ASU-W2V-Workshop-master/GoogleNews-vectors-negative300.bin.gz', binary=True)

<h1><center>Explore Word2Vec Similarity</center></h1>

In [11]:
model.vector_size

300

In [5]:
#vocabulary words
#model.index2word

In [6]:
#model.get_vector('dog')
model['dog']

array([-1.1043e-01,  8.1217e-01,  7.3668e-02,  1.9023e-01, -5.2888e-02,
        6.1468e-02,  1.6076e-01,  4.1302e-01, -3.0199e-01, -9.0827e-01,
        2.7504e-01, -3.1890e-02, -2.8842e-01,  2.3447e-01,  4.7679e-01,
        5.0124e-01,  2.9371e-01,  2.7029e-01,  5.4745e-02,  9.8038e-02,
        5.7116e-01,  3.6755e-01,  4.0734e-02,  3.4347e-01, -1.8256e-01,
       -2.8935e-01,  2.3826e-02, -1.9401e-01,  2.4444e-01,  1.3407e-01,
       -1.6494e-01, -2.6983e-01, -2.6234e-01, -2.1779e-01, -8.7528e-01,
        7.3822e-01, -8.7931e-02, -1.0876e-02, -2.6540e-01,  3.4668e-01,
       -5.5814e-01,  1.7591e-01,  1.6926e-01, -1.5725e-01, -5.0430e-01,
       -2.0100e-01,  6.6701e-01, -3.2518e-02,  4.5012e-02,  6.5675e-02,
       -1.6061e-01, -7.3363e-01,  2.4642e-01,  3.4325e-01,  2.1899e-01,
        4.8646e-02, -5.9987e-01, -5.8153e-02, -5.1694e-02, -5.7846e-01,
        3.0000e-01,  3.5078e-01,  4.6646e-01, -7.5309e-03,  1.0455e-01,
       -5.1016e-01, -5.5987e-02, -1.0295e-01, -2.6476e-01, -4.12

In [16]:
model.similar_by_word('dog')

[('dogs', 0.8680490255355835),
 ('puppy', 0.8106428384780884),
 ('pit_bull', 0.780396044254303),
 ('pooch', 0.7627376317977905),
 ('cat', 0.7609457969665527),
 ('golden_retriever', 0.7500901818275452),
 ('German_shepherd', 0.7465174198150635),
 ('Rottweiler', 0.7437615394592285),
 ('beagle', 0.7418621778488159),
 ('pup', 0.7406911253929138)]

In [8]:
model.similarity('man','boy')

0.62082785

In [9]:
model.doesnt_match(['banana', 'apple', 'grapes', 'egg'])

'egg'

### Word2Vec (Neural Net)

a. Skip Gram Model: use context words to predict target word

b. Continuous Bag of Words Model: use target word to predict context words



<h1><center>Read in Movie Review Data</center></h1>

In [69]:
import numpy as np
import pandas as pd

In [71]:
# read the data into a pandas dataframe
import pandas as pd
import os
def data2df (path, label):
    file, text = [], []
    for f in os.listdir(path):
        file.append(f)
        fhr = open(path+f, 'r') 
        t = fhr.read()
        text.append(t)
        fhr.close()
    return(pd.DataFrame({'file': file, 'text': text, 'class':label}))

dfneg = data2df('Data/MoviePosNeg/neg/', 0) # NEG
dfpos = data2df('Data/MoviePosNeg/pos/', 1) # POS

df = pd.concat([dfpos, dfneg], axis=0)
df.sample(frac=0.005)

file  \
791  cv590_20712.txt   
64   cv810_12458.txt   
981  cv855_22134.txt   
175   cv909_9960.txt   
855  cv209_28973.txt   
263  cv003_12683.txt   
421  cv116_28942.txt   
629  cv728_17931.txt   
340  cv725_10266.txt   
392  cv259_10934.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

<h1><center>Clean& Preprocess</center></h1>

In [72]:
def preprocess(text):
    # replace one or more white-space characters with a space
    regex = re.compile(r"\s+")                               
    text = regex.sub(' ', text)    
    # lower case
    text = text.lower()          
    # remove digits and punctuation
    regex = re.compile(r"[%s%s]" % (string.punctuation, string.digits))
    text = regex.sub(' ', text)           
    # remove stop words
    sw = stopwords.words('english')
    text = text.split()                                              
    text = ' '.join([w for w in text if w not in sw]) 
    # remove short words
    ' '.join([w for w in text.split() if len(w) >= 2])
    # lemmatize
    text = ' '.join([(WordNetLemmatizer()).lemmatize(w) for w in text.split()]) 
    return text

In [73]:
df['text_tokenize'] = df.apply(lambda x: word_tokenize(str(x['text'])),axis=1)

In [74]:
reviews = df['text_tokenize'].values.tolist()

In [75]:
reviews_no_stop = []
for review in reviews:
    temp_lst = [w for w in review if not w in stop_words] 
    reviews_no_stop.append(temp_lst)

In [76]:
df['text_tokenize'] = reviews_no_stop

<h1><center>Train Model Using Word2Vec</center></h1>

In [77]:
from gensim.models import word2vec
w2v_model = word2vec.Word2Vec(reviews_no_stop, size = 300) 

In [78]:
def vectorize(model):
    review_counter = 0 
    all_review_vectors = np.zeros((len(reviews),300))
    all_words = set(model.wv.index2word)

    for review in reviews:
        word_counter = 0
        one_review_vector = np.zeros(300)

        for word in review: 
            if word in all_words: 
                word_counter = word_counter + 1
                one_review_vector = np.add(one_review_vector,model[word])

        one_review_vector = np.divide(one_review_vector, word_counter)        

        all_review_vectors[review_counter] = one_review_vector
        review_counter = review_counter + 1 

    # set null values to be zero 
    all_review_vectors = np.nan_to_num(all_review_vectors)
    return all_review_vectors

In [79]:
all_review_vectors_w2v_model = vectorize(w2v_model)

In [80]:
def logistic_model_accuracy(vectors):
    # split dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(vectors,df['class'],
                                                       test_size = 0.2, random_state = 0)

    # initialize prediction regression model 
    logistic_regression_model = LogisticRegression()

    # fit model 
    logistic_regression_model.fit(X_train, y_train)

    # predict y values for model 
    y_train_pred = logistic_regression_model.predict(X_train)

    return ("Accuracy Score: ", round(accuracy_score(y_train, y_train_pred),2))

In [81]:
logistic_model_accuracy(all_review_vectors_w2v_model)

('Accuracy Score: ', 0.62)

<h1><center> Use Pre-Trained Word2Vec Model <h1><center>

In [82]:
model

In [84]:
print(model.most_similar(positive=['man', 'queen'], negative=['king'])[0])

('woman', 0.7609435319900513)


In [85]:
all_review_vectors_pretrained_model = vectorize(model)

In [86]:
logistic_model_accuracy(all_review_vectors_pretrained_model)

('Accuracy Score: ', 0.76)

<h1><center>  Tfidf + Logistic Reg</center></h1>

In [87]:
X_train, X_test, y_train, y_test  = train_test_split(df['text'],df['class'])

In [90]:
vect = TfidfVectorizer()
vect.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [91]:
vect.transform(X_train)

<1500x35257 sparse matrix of type '<class 'numpy.float64'>'
	with 498382 stored elements in Compressed Sparse Row format>

In [93]:
sdg_classifier = SGDClassifier()
sdg_classifier.fit(vect.transform(X_train), y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [94]:
y_pred_tfidf = sdg_classifier.predict(vect.transform(X_test))

In [97]:
print("TFIDF Accuracy Score: ", round(accuracy_score(y_pred_tfidf,y_test),2))

TFIDF Accuracy Score:  0.84


In [99]:
from sklearn.linear_model import LogisticRegression
mdl = LogisticRegression()
mdl.fit(vect.transform(X_train), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [103]:
XTtest = pd.DataFrame(vect.transform(X_test).toarray(), columns=vect.get_feature_names())
XTtest.head()

,00,000,0009f,007,00s,05,10,100,1000,10000,...,zucker,zuehlke,zuko,zulu,zundel,zurg,zus,zwick,zwigoff,zycie
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
ypred = mdl.predict(XTtest)
from sklearn import metrics
print (metrics.accuracy_score(y_test, ypred))
print (metrics.confusion_matrix(y_test, ypred))
print (metrics.classification_report(y_test, ypred))

0.83
[[204  38]
 [ 47 211]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.83       242
           1       0.85      0.82      0.83       258

   micro avg       0.83      0.83      0.83       500
   macro avg       0.83      0.83      0.83       500
weighted avg       0.83      0.83      0.83       500



<h1><center> TFIDF + W2V + Logistic Reg</center></h1>

In [106]:
def avgW2V(model, df, weighted=True):
    modeldim = model.vector_size      # test: 3
    modelvocab = model.index2word # test: modelvocab = ['w1', 'w2', 'w3', 'w4', 'w5']
    numdocs = (len(df))
    avgw2v = pd.DataFrame(np.zeros((numdocs,modeldim)))
    # for each doc 
    for i in range(numdocs):
        # pick up only those words that have values > 0 in df
        ser = df.loc[i][df.loc[i]>0] 
        words, scores = ser.index, ser.values
        #  if weighted=False , set scores to 1, else pick up the real scores
        if (weighted==False):
            scores = scores/scores
        # find the avg w2v doc vectors
        sumw2v, sumweights = 0, 0
        for s,w in zip(scores,words):
            if w in modelvocab:
                sumw2v += s*model[w]
                sumweights += s
        avgw2v.loc[i] = sumw2v/sumweights
    return avgw2v

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(
    use_idf=True, smooth_idf=True, norm='l2',
    min_df=3, max_df=1.0, max_features=5000, 
    ngram_range=(1, 1))
XTtrain = pd.DataFrame(tv.fit_transform(X_train).toarray(), columns=tv.get_feature_names())
XTtrain.head()

,000,10,100,101,11,12,13,13th,14,15,...,younger,your,yourself,youth,zany,zellweger,zero,zeta,zone,zwick
0,0.0,0.0,0.0,0.0,0.033167,0.059063,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.014889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
aw2vXTtrain = avgW2V(model, XTtrain, weighted=True)
aw2vXTtrain.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.051567,0.056669,-0.012448,0.087815,-0.041689,-0.004975,0.029989,-0.086169,0.051148,0.082134,...,-0.087501,0.019493,-0.064522,0.018563,-0.031242,-0.007383,0.016751,-0.043850,0.019675,0.005698
1,0.054291,0.040925,0.014341,0.087313,-0.039136,-0.015663,0.008342,-0.111436,0.094570,0.069755,...,-0.037187,-0.014824,-0.066222,0.026243,-0.048340,-0.026352,-0.033588,-0.070457,0.033098,-0.000831
2,0.055673,0.033510,0.027125,0.078745,-0.048241,0.009528,0.060278,-0.057746,0.069177,0.088370,...,-0.062296,0.013395,-0.083208,0.032818,-0.059708,-0.022631,0.008184,-0.075018,0.028616,0.003475
3,0.021245,0.041555,0.014672,0.101468,-0.051515,-0.010543,0.019666,-0.073800,0.062720,0.078564,...,-0.051025,0.038158,-0.078457,0.028324,-0.031824,-0.022925,-0.004135,-0.042222,0.008746,0.000337
4,0.031039,0.045775,0.009521,0.089179,-0.048876,-0.016164,0.031938,-0.055786,0.086130,0.085161,...,-0.088386,0.042345,-0.089749,0.040763,-0.047209,-0.035047,0.015346,-0.038481,0.046984,-0.006512


In [111]:
from sklearn.linear_model import LogisticRegression
mdl = LogisticRegression()
mdl.fit(aw2vXTtrain, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [112]:
aw2vXTtest = avgW2V(model, XTtest, weighted=True)
aw2vXTtest.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.046520,0.024911,0.003000,0.105945,-0.042543,0.011471,0.017052,-0.081678,0.055745,0.076509,...,-0.041567,0.028946,-0.106929,0.024135,-0.042407,0.007285,-0.007538,-0.067152,0.024045,-0.006080
1,0.077593,0.052294,0.020784,0.076122,-0.052259,0.008913,0.038178,-0.076457,0.059356,0.072469,...,-0.084569,0.025149,-0.081766,0.006284,-0.033267,-0.023910,0.009665,-0.050233,0.019294,0.008253
2,0.042815,0.057870,0.022324,0.097939,-0.046625,0.010425,0.058178,-0.071364,0.060803,0.084427,...,-0.071418,0.041894,-0.068603,0.023713,-0.042947,-0.022485,0.003380,-0.046121,0.013674,0.008507
3,0.029677,0.055065,0.041357,0.056014,-0.067226,-0.016346,0.010914,-0.052221,0.049772,0.039157,...,-0.054555,0.021470,-0.081703,0.040457,-0.043773,-0.020818,0.007604,-0.054513,0.027563,0.023348
4,0.020634,0.052961,0.007279,0.070402,-0.033430,0.001643,0.023190,-0.078544,0.055775,0.067468,...,-0.052076,0.034935,-0.065114,0.019398,-0.037293,-0.025797,-0.009589,-0.058464,0.010734,0.038726


In [114]:
ypred = mdl.predict(aw2vXTtest)
from sklearn import metrics
print (metrics.accuracy_score(y_test, ypred))
print (metrics.confusion_matrix(y_test, ypred))
print (metrics.classification_report(y_test, ypred))

0.762
[[195  47]
 [ 72 186]]
              precision    recall  f1-score   support

           0       0.73      0.81      0.77       242
           1       0.80      0.72      0.76       258

   micro avg       0.76      0.76      0.76       500
   macro avg       0.76      0.76      0.76       500
weighted avg       0.77      0.76      0.76       500

